<a href="https://colab.research.google.com/github/butchland/fastai_xla_extensions/blob/master/explore_nbs/Demo_batch_tfms_GPU_performance_diff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl -s https://course.fast.ai/setup/colab | bash

Updating fastai...
Done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### [RUNME] Install Colab compatible PyTorch/XLA wheels and dependencies
This may take up to ~2 minutes



In [ ]:
# VERSION = "20200516"  #@param ["1.5" , "20200516", "nightly"]
# !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# !python pytorch-xla-env-setup.py --version $VERSION

In [ ]:
!pip install fastai2 --upgrade > /dev/null
!pip freeze | grep fastai2

fastai2==0.0.17


In [ ]:
!pip install git+https://github.com/butchland/fastai_xla_extensions > /dev/null
!pip freeze | grep fastai_xla_extensions

  Running command git clone -q https://github.com/butchland/fastai_xla_extensions /tmp/pip-req-build-rsot5oqn


In [ ]:
import fastai_xla_extensions.core

In [ ]:
from fastai2.vision.all import *

In [ ]:
path = untar_data(URLs.MNIST)

In [ ]:
Path.BASE_PATH = path; path.ls()

(#2) [Path('training'),Path('testing')]

In [ ]:
datablock = DataBlock(
    blocks=(ImageBlock,CategoryBlock),
    get_items=get_image_files,
    get_y=parent_label,
    splitter=GrandparentSplitter(train_name='training',valid_name='testing'),
    item_tfms=Resize(28),
    batch_tfms=[]
)

In [ ]:
datablock_tfms= datablock.new(batch_tfms=[*aug_transforms(size=28, do_flip=False), Normalize],item_tfms=Resize(40))

In [ ]:
datablock_tfms.summary(path)

Setting-up type transforms pipelines
Found 70000 items
2 datasets of sizes 60000,10000
Setting up Pipeline: PILBase.create
Setting up Pipeline: parent_label -> Categorize

Building one sample
  Pipeline: PILBase.create
    starting from
      /root/.fastai/data/mnist_png/training/0/38706.png
    applying PILBase.create gives
      PILImage mode=RGB size=28x28
  Pipeline: parent_label -> Categorize
    starting from
      /root/.fastai/data/mnist_png/training/0/38706.png
    applying parent_label gives
      0
    applying Categorize gives
      TensorCategory(0)

Final sample: (PILImage mode=RGB size=28x28, TensorCategory(0))


Setting up after_item: Pipeline: Resize -> ToTensor
Setting up before_batch: Pipeline: 
Setting up after_batch: Pipeline: IntToFloatTensor -> AffineCoordTfm -> LightingTfm -> Normalize

Building one batch
Applying item_tfms to the first sample:
  Pipeline: Resize -> ToTensor
    starting from
      (PILImage mode=RGB size=28x28, TensorCategory(0))
    applying R

In [ ]:
dls1 = datablock_tfms.dataloaders(path,bs=256)
dls1.device

device(type='cuda', index=0)

In [ ]:
learn1 = cnn_learner(dls1, resnet18, metrics=accuracy)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth


In [ ]:
learn1.fine_tune(0, freeze_epochs=4)

epoch,train_loss,valid_loss,accuracy,time
0,1.570261,0.898596,0.709400,01:40
1,0.966967,0.602850,0.801600,01:39
2,0.659274,0.414949,0.866800,01:38
3,0.458183,0.269513,0.910900,01:37


In [ ]:
datablock_no_tfms= datablock.new(batch_tfms=[],item_tfms=Resize(28))

In [ ]:
datablock_no_tfms.summary(path)

Setting-up type transforms pipelines
Found 70000 items
2 datasets of sizes 60000,10000
Setting up Pipeline: PILBase.create
Setting up Pipeline: parent_label -> Categorize

Building one sample
  Pipeline: PILBase.create
    starting from
      /root/.fastai/data/mnist_png/training/0/38706.png
    applying PILBase.create gives
      PILImage mode=RGB size=28x28
  Pipeline: parent_label -> Categorize
    starting from
      /root/.fastai/data/mnist_png/training/0/38706.png
    applying parent_label gives
      0
    applying Categorize gives
      TensorCategory(0)

Final sample: (PILImage mode=RGB size=28x28, TensorCategory(0))


Setting up after_item: Pipeline: Resize -> ToTensor
Setting up before_batch: Pipeline: 
Setting up after_batch: Pipeline: IntToFloatTensor

Building one batch
Applying item_tfms to the first sample:
  Pipeline: Resize -> ToTensor
    starting from
      (PILImage mode=RGB size=28x28, TensorCategory(0))
    applying Resize gives
      (PILImage mode=RGB size=28x2

In [ ]:
dls2 = datablock_no_tfms.dataloaders(path,bs=256)
dls2.device

device(type='cuda', index=0)

In [ ]:
learn2 = cnn_learner(dls2, resnet18, metrics=accuracy)

In [ ]:
learn2.fine_tune(0, freeze_epochs=4)

epoch,train_loss,valid_loss,accuracy,time
0,1.449692,0.844848,0.735000,01:33
1,0.875995,0.559240,0.822400,01:33
2,0.555925,0.381971,0.877700,01:32
3,0.369941,0.256615,0.918100,01:33
